##Importing 라이브러리

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)  # 무조건 42로 세팅!!

In [7]:
!pip install xgboost

In [8]:
from xgboost import XGBClassifier

##파일 불러오기

In [328]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
submission = pd.read_csv("/content/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

(48199, 13) (7695, 12) (7695, 2)


##데이터 전처리 (Train + Test)

###0)불필요 열 제거

In [329]:
train.drop(columns=['click_id', 'session ID'], inplace=True)

In [330]:
test.drop(columns=['click_id', 'session ID'], inplace=True)

###1) main category 결측치 채우기

In [331]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (train["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
train.loc[mask, "page 1 (main category)"] = train.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(train.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])
#page 2 (clothing model) 삭제
train.drop(columns=['page 2 (clothing model)'], inplace=True)

31


In [332]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (test["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
test.loc[mask, "page 1 (main category)"] = test.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(test.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])
#page 2 (clothing model) 삭제
test.drop(columns=['page 2 (clothing model)'], inplace=True)

3


###2)시간과 관련한 데이터 추출

In [333]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train.drop(columns=['date'], inplace=True)

In [334]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test.drop(columns=['date'], inplace=True)

###3)colour

In [335]:
# 색상 그룹 매핑 (높은 Click 비율 → 낮은 숫자)
color_map = {
    2: 0, 3: 0, 8: 0,  # 가장 높은 Click 비율 (가장 낮은 숫자)
    4: 1, 7: 1, 14: 1,  # 두 번째 그룹
    9: 2, 10: 2, 11: 2, 13: 2,  # 세 번째 그룹
    1: 3, 5: 3, 6: 3, 12: 3  # 가장 낮은 Click 비율 (가장 높은 숫자)
}

# colour 컬럼 변환
train["colour"] = train["colour"].map(color_map)

In [336]:
test["colour"] = test["colour"].map(color_map)

###4)country target encoding

In [337]:
train["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

In [338]:
test["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

###5)price log변환

In [339]:
train['log_price'] = np.log1p(train['price'])

In [340]:
test['log_price'] = np.log1p(test['price'])

###6)model photography target_encoding&

In [341]:
train['model photography_ctr'] = train.groupby('model photography')['Clicked'].transform('mean')

In [342]:
test['model photography_ctr'] = train.groupby('model photography')['Clicked'].transform('mean')

###7)중복되거나 중요성 떨어지는 column drop

In [343]:
train.drop(columns=['country','year', 'month','model photography'], inplace=True)

In [344]:
test.drop(columns=['country','year', 'month','model photography'], inplace=True)

In [345]:
train

order  page 1 (main category)  colour  location  page  price  Clicked  \
0          1                     1.0       3         5     1   39.0        1   
1          2                     1.0       3         6     1   40.0        0   
2          3                     2.0       2         2     1   46.0        1   
3          4                     2.0       3         6     1   30.0        0   
4          5                     2.0       1         3     1   41.0        0   
...      ...                     ...     ...       ...   ...    ...      ...   
48194     38                     2.0       3         5     2   39.0        0   
48195     39                     2.0       0         6     2   46.0        0   
48196     40                     4.0       1         2     1   49.0        1   
48197     41                     4.0       3         1     5   49.0        0   
48198      1                     1.0       0         2     1   45.0        0   

       day  weekday  country_encoded  log_price  model photography_ctr  
0        1        0          0.36522   3.688879               0.374710  
1        1        0          0.36522   3.713572               0.374710  
2        1        0          0.36522   3.850148               0.374710  
3        1        0          0.36522   3.433987               0.305184  
4        1        0          0.36522   3.737670               0.305184  
...    ...      ...              ...        ...                    ...  
48194   30        1          0.36522   3.688879               0.305184  
48195   30        1          0.36522   3.850148               0.374710  
48196   30        1          0.36522   3.912023               0.374710  
48197   30        1          0.36522   3.912023               0.305184  
48198   30        1          0.36522   3.828641               0.305184  

[48199 rows x 12 columns]

##학습하기
-XGboost 모델 사용하기
-하이퍼파라미터의 영향을 많이 받는 모델이기에 최적의 조합 찾기

In [346]:
X = train.drop('Clicked', axis=1)
y = train['Clicked']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(38559, 11) (9640, 11) (38559,) (9640,)


-

In [347]:
clf=XGBClassifier()
params = {'n_estimators':[50, 100, 200],
          'max_depth':[2,3,4],
          'learning_rate':[0.01, 0.05, 0.1]
          }
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200}
최고 예측 정확도: 0.6817


In [ ]:
#전거 = [0.6844-0.6790]

In [362]:
xgb_clf = XGBClassifier(**grid_cv.best_params_)
#[0.6939-0.6647],[]
#xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.05)
xgb_clf.fit(X_train, y_train)

train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6831
Val ACC : 0.6794


In [364]:
xgb_clf = XGBClassifier(n_estimators=150, max_depth=2, learning_rate = 0.2)
xgb_clf.fit(X_train, y_train)
#[0.6842-0.6825]0.2
train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6842
Val ACC : 0.6825


In [365]:
feature_importance = pd.DataFrame(xgb_clf.feature_importances_.reshape((1, -1)), columns=X_train.columns, index=['feature_importance'])
feature_importance

order  page 1 (main category)    colour  location  \
feature_importance  0.090047                 0.19088  0.386799  0.096493   

                       page     price      day   weekday  country_encoded  \
feature_importance  0.10097  0.029703  0.00808  0.010565         0.011478   

                    log_price  model photography_ctr  
feature_importance        0.0               0.074984

##submission.csv 생성

In [360]:
X_test = test

output = xgb_clf.predict(X_test)
assert len(output) == 7695  # sanity check

In [361]:
submission['Clicked'] = output
submission.to_csv('submission.csv', index=False)